# Part 2 - Data Cleaning

In [1]:
# Loading libraries:

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
# Reading the new csv files:

rounds = pd.read_csv("rounds_clean.csv",encoding="ISO-8859-1")
companies = pd.read_csv("companies_clean.csv",sep="\t",encoding="ISO-8859-1")

In [3]:
# Let us quickly see the unique values once again in the two files

print(len(companies["permalink"].unique()))
print(len(rounds["company_permalink"].unique()))

66368
66368


### Missing Value Treatment

In [4]:
# Companies dataframe:

companies.isnull().sum()

permalink            0
name                 1
homepage_url      5058
category_list     3148
status               0
country_code      6958
state_code        8547
region            8030
city              8028
founded_at       15221
dtype: int64

In [5]:
# Rounds data:

rounds.isnull().sum()

company_permalink              0
funding_round_permalink        0
funding_round_type             0
funding_round_code         83809
funded_at                      0
raised_amount_usd          19990
dtype: int64

Observation: 

   The permalink and company_permalink are unique ids and there are no missing values in those columns.We can merge both the 
tables and make one master table.

In [6]:
# Merging the two dataframes:

master = pd.merge(companies,rounds,how="inner",left_on="permalink",right_on="company_permalink")
master.head()

,permalink,name,homepage_url,category_list,status,country_code,state_code,region,city,founded_at,company_permalink,funding_round_permalink,funding_round_type,funding_round_code,funded_at,raised_amount_usd
0,/organization/-fame,#fame,http://livfame.com,Media,operating,IND,16,Mumbai,Mumbai,NaN,/organization/-fame,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,B,05-01-2015,10000000.0
1,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,A,14-10-2014,NaN
2,/organization/-qounter,:Qounter,http://www.qounter.com,Application Platforms|Real Time|Social Network...,operating,USA,DE,DE - Other,Delaware City,04-09-2014,/organization/-qounter,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,NaN,01-03-2014,700000.0
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",http://oneofthem.jp,Apps|Games|Mobile,operating,NaN,NaN,NaN,NaN,NaN,/organization/-the-one-of-them-inc-,/funding-round/650b8f704416801069bb178a1418776b,venture,B,30-01-2014,3406878.0
4,/organization/0-6-com,0-6.com,http://www.0-6.com,Curated Web,operating,CHN,22,Beijing,Beijing,01-01-2007,/organization/0-6-com,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,A,19-03-2008,2000000.0


Since permalink and company_permalink are the same, let us remove one of the columns from the master dataset

In [7]:
master = master.drop(["company_permalink"],axis=1)

In [8]:
# Print the column names:

master.columns

Index(['permalink', 'name', 'homepage_url', 'category_list', 'status',
       'country_code', 'state_code', 'region', 'city', 'founded_at',
       'funding_round_permalink', 'funding_round_type', 'funding_round_code',
       'funded_at', 'raised_amount_usd'],
      dtype='object')

In [9]:
# Column wise missing values:

master.isnull().sum()

permalink                      0
name                           1
homepage_url                6134
category_list               3408
status                         0
country_code                8676
state_code                 10944
region                     10165
city                       10162
founded_at                 20521
funding_round_permalink        0
funding_round_type             0
funding_round_code         83807
funded_at                      0
raised_amount_usd          19989
dtype: int64

In [10]:
# Lets look at the fraction of missing values:

round((master.isnull().sum()/len(master))*100,2)

permalink                   0.00
name                        0.00
homepage_url                5.34
category_list               2.96
status                      0.00
country_code                7.55
state_code                  9.52
region                      8.84
city                        8.84
founded_at                 17.85
funding_round_permalink     0.00
funding_round_type          0.00
funding_round_code         72.91
funded_at                   0.00
raised_amount_usd          17.39
dtype: float64

Observation:
1. There are about 73% of the null values present in the funding_round_code column
2. Taking into consideration the business objective,the columns homepage_url,founded_at,state_code,region and city need not be used.

In [11]:
# Dropping the unnecessary columns:

master = master.drop(["funding_round_code","homepage_url","founded_at","state_code","region","city"],axis=1)

In [12]:
master.head()

,permalink,name,category_list,status,country_code,funding_round_permalink,funding_round_type,funded_at,raised_amount_usd
0,/organization/-fame,#fame,Media,operating,IND,/funding-round/9a01d05418af9f794eebff7ace91f638,venture,05-01-2015,10000000.0
1,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,operating,USA,/funding-round/22dacff496eb7acb2b901dec1dfe5633,venture,14-10-2014,NaN
2,/organization/-qounter,:Qounter,Application Platforms|Real Time|Social Network...,operating,USA,/funding-round/b44fbb94153f6cdef13083530bb48030,seed,01-03-2014,700000.0
3,/organization/-the-one-of-them-inc-,"(THE) ONE of THEM,Inc.",Apps|Games|Mobile,operating,NaN,/funding-round/650b8f704416801069bb178a1418776b,venture,30-01-2014,3406878.0
4,/organization/0-6-com,0-6.com,Curated Web,operating,CHN,/funding-round/5727accaeaa57461bd22a9bdd945382d,venture,19-03-2008,2000000.0


In [13]:
# Checking for the null values :

(round((master.isnull().sum()/len(master))*100),2)

(permalink                   0.0
 name                        0.0
 category_list               3.0
 status                      0.0
 country_code                8.0
 funding_round_permalink     0.0
 funding_round_type          0.0
 funded_at                   0.0
 raised_amount_usd          17.0
 dtype: float64,
 2)

Raised amount USD is extremely important for our analysis and that needs to be carefully treated as there are 17% null values 
in the data. Also, country-code will be used for country-wise analysis and category list will be used to merge the dataframe with the main categories.

In [14]:
# Let us see how we can deal with the missing values in raised_amount_usd

master["raised_amount_usd"].describe()

count    9.495700e+04
mean     1.042704e+07
std      1.148225e+08
min      0.000000e+00
25%      3.225000e+05
50%      1.680511e+06
75%      7.000000e+06
max      2.127194e+10
Name: raised_amount_usd, dtype: float64

Observation:
1. The mean and median of the raised amount are not close to each other in values.The mean is USD 10 million whereas the median is only USD 1 million.
2. Also, minimum and maximum values are miles apart.

Treatment to be made:
1. In general, since there is a huge spread in the funding amounts, it will be inappropriate to impute it with the metric such as median or mean.
2. Also,since we have quite large number of observations, it is wiser to just drop the rows.


In [15]:
# Removing NaN values in the raised amount usd:

master = master[~np.isnan(master["raised_amount_usd"])]

In [16]:
# Checking for the null values for raised amount USD:

round(master.isnull().sum()/(len(master))*100,2)

permalink                  0.00
name                       0.00
category_list              1.10
status                     0.00
country_code               6.16
funding_round_permalink    0.00
funding_round_type         0.00
funded_at                  0.00
raised_amount_usd          0.00
dtype: float64

In [17]:
# Now lets look at the column country_code. To see the distribution of the categorical variables,it is best to convert them into
#category type.

country_codes = master["country_code"].astype("category")

In [18]:
# Displaying frequencies of each category:

country_codes.value_counts()

USA    62049
GBR     5019
CAN     2616
CHN     1927
IND     1649
       ...  
SYC        1
KNA        1
SEN        1
LAO        1
PRY        1
Name: country_code, Length: 134, dtype: int64

By far,the most number of investments have happened in American countries. We can also see the fraction.

In [19]:
# Viewing the fraction of counts of country codes:

(master["country_code"].value_counts()/len(master)*100)

USA    65.344314
GBR     5.285550
CAN     2.754931
CHN     2.029340
IND     1.736576
         ...    
HND     0.001053
GGY     0.001053
TGO     0.001053
MNE     0.001053
SEN     0.001053
Name: country_code, Length: 134, dtype: float64

Now we can either delete the rows having country codes missing (about 6%), or we can impute them by USA. Since the number of missing values is quite small and there is decent amount of data, it may be better to just remove the rows. 

In [20]:
# Let us remove the rows 

master = master[~pd.isnull(master["country_code"])]

In [21]:
# Check for the null values :

(master.isnull().sum()/len(master))*100

permalink                  0.000000
name                       0.001122
category_list              0.649781
status                     0.000000
country_code               0.000000
funding_round_permalink    0.000000
funding_round_type         0.000000
funded_at                  0.000000
raised_amount_usd          0.000000
dtype: float64

Note that the fraction of missing values in the remaining dataframe has also reduced now - only 0.65% in category_list. Lets 
remove all those as well.

Note : Removing the missing values is completely optional. There is nothing wrong with that. But since we will use that column 
for merging with the "main_categories", removing the missing values will be quite convenient and again we have enough data.

In [22]:
# Removing rows with missing category_list values:

master = master[~pd.isnull(master["category_list"])]

In [24]:
# Checking for the null values:

(round(master.isnull().sum()/len(master))*100,2)

(permalink                  0.0
 name                       0.0
 category_list              0.0
 status                     0.0
 country_code               0.0
 funding_round_permalink    0.0
 funding_round_type         0.0
 funded_at                  0.0
 raised_amount_usd          0.0
 dtype: float64,
 2)

There are no null values in the dataset,so this is the cleaned dataset

In [25]:
# Let us save this cleaned data:

master.to_csv("master_df.csv",sep=",",index=False)

In [27]:
# Using master_df dataset for further analysis:

# Information of the data:

master.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 88528 entries, 0 to 114944
Data columns (total 9 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   permalink                88528 non-null  object 
 1   name                     88527 non-null  object 
 2   category_list            88528 non-null  object 
 3   status                   88528 non-null  object 
 4   country_code             88528 non-null  object 
 5   funding_round_permalink  88528 non-null  object 
 6   funding_round_type       88528 non-null  object 
 7   funded_at                88528 non-null  object 
 8   raised_amount_usd        88528 non-null  float64
dtypes: float64(1), object(8)
memory usage: 6.8+ MB


In [28]:
# Checking data retained after treating for the missing values:

100*(len(master)/len(rounds))

77.01502405414575

77% of the data is retained after handling the missing values.